In [178]:
import sys
sys.path.append("..")
from utils.utils import Matrix, Vec
import numpy as np
from itertools import islice

In [179]:
with open("input.txt") as file:
    codes = file.read().splitlines()
    
numpad = Matrix.from_str("789\n456\n123\nX0A")
dirpad = Matrix.from_str("X^A\n<v>")

In [180]:
from functools import cache
from tqdm import tqdm
import math

In [181]:
dirmap = {
    '>':Vec(1,0),
    '<':Vec(-1,0),
    '^':Vec(0,-1),
    'v':Vec(0,1)
}

In [183]:
@cache
def check_x_in_path(pad, pos, path):
    for move in path:
        pos += dirmap[move]
        if pad[pos] == 'X':
            return True
    return False

@cache
def vec_to_path(v, get_all=True): 
    # kann nur 2 kürzeste geben. davon kann 1 kaputt sein. welcher kürzer ist erst später ersichtlich?
    # reihenfolge >^v< sorgt für nicht kaputt aber evtl nicht der kürzeste
    
    if get_all:
        ret = []
        xchar = '>'*abs(v.x) if v.x>0 else '<'*abs(v.x)
        ychar = 'v'*abs(v.y) if v.y>0 else '^'*abs(v.y)
        return [xchar+ychar, ychar+xchar]
    ret = ""
    # if x <= v and Vec(0, 0) <= x: # x liegt auf pfad evtl
    #     if v.x > 0:
    #         ret += '>' * v.x
    #     if v.y < 0:
    #         ret += '^' * abs(v.y)
    #     if v.x < 0:
    #         ret += '<' * abs(v.x)
    #     if v.y > 0:
    #         ret += 'v' * v.y
    # else:
    if v.x > 0:
        ret += '>' * v.x
    if v.y > 0:
        ret += 'v' * v.y
    if v.y < 0:
        ret += '^' * abs(v.y)
    if v.x < 0:
        ret += '<' * abs(v.x)
        
    return [ret]

@cache
def get_instructions_moves(pad, start, end):
    # print(start,end)
    sp = Vec(*list(zip(*np.where(pad==start)))[0])
    ep = Vec(*list(zip(*np.where(pad==end)))[0])
    dist = ep - sp
    get_all = pad.shape == numpad.shape
    return [path for path in vec_to_path(dist, get_all) if not check_x_in_path(pad, sp, path)]
    
@cache
def get_instructions(code, depth):
    """get instruction set that causes robot to type Code"""
    print(depth)
    if code[0].isnumeric():
        pad = numpad
    else:
        pad = dirpad
    instructions = [""]
    code = 'A' + code
    for start, end in zip(code, code[1:]):
        # print(start,end)
        # instructions += get_instructions_moves(pad, start, end) + 'A'
        new_instructions = get_instructions_moves(pad, start, end)
        new_instructionset = []
        for instruction in instructions:
            for new_instruction in new_instructions:
                new_instructionset.append(instruction + new_instruction + 'A')
            instructions = new_instructionset
    instructions = list(set(instructions))
    # print(depth, instructions)
    if depth == 0:
        return min(map(len, instructions))
    else:
        return min([get_instructions(instruction, depth-1) for instruction in instructions])

@cache
def get_complexity(code, intermediate_pads = 2):
    l1 = get_instructions('A' + code, intermediate_pads)
    return l1
    # return shortest * int(code[:-1]), shortest, code

In [184]:
get_instructions("980A", 25)

25
24
23
22
21
20
19
18
17
16
15
14
13
12


KeyboardInterrupt: 

In [37]:
s=0
for code in codes:
    comp, l, c = get_complexity(code, 25)
    print(comp, l, c)
    s+=comp
s

A 2
2 8
8 6
6 A
A ^


IndexError: list index out of range

In [500]:
# 374412 too low

In [ ]:
@cache
def window(seq, n=2):
    "Returns a sliding window (of width n and step 1) over data from the iterable"
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

@cache
def check_x_in_path(pad, pos, path):
    for move in path:
        pos += dirmap[move]
        if pad[pos] == 'X':
            return True
    return False

@cache
def vec_to_path(v, get_all=True): 
    # kann nur 2 kürzeste geben. davon kann 1 kaputt sein. welcher kürzer ist erst später ersichtlich?
    # reihenfolge >^v< sorgt für nicht kaputt aber evtl nicht der kürzeste
    
    if get_all:
        ret = []
        xchar = '>'*abs(v.x) if v.x>0 else '<'*abs(v.x)
        ychar = 'v'*abs(v.y) if v.y>0 else '^'*abs(v.y)
        return [xchar+ychar, ychar+xchar]
    ret = ""
    # if x <= v and Vec(0, 0) <= x: # x liegt auf pfad evtl
    #     if v.x > 0:
    #         ret += '>' * v.x
    #     if v.y < 0:
    #         ret += '^' * abs(v.y)
    #     if v.x < 0:
    #         ret += '<' * abs(v.x)
    #     if v.y > 0:
    #         ret += 'v' * v.y
    # else:
    if v.x > 0:
        ret += '>' * v.x
    if v.y > 0:
        ret += 'v' * v.y
    if v.y < 0:
        ret += '^' * abs(v.y)
    if v.x < 0:
        ret += '<' * abs(v.x)
        
    return [ret]

@cache
def get_instructions_moves(pad, start, end):
    sp = Vec(*list(zip(*np.where(pad==start)))[0])
    ep = Vec(*list(zip(*np.where(pad==end)))[0])
    dist = ep - sp
    get_all = pad.shape == numpad.shape
    return [path for path in vec_to_path(dist, get_all) if not check_x_in_path(pad, sp, path)]
    
@cache
def get_instructions(pad, code):
    """get instruction set that causes robot to type Code"""
    instructions = [""]
    for start, end in window(code):
        # print(start,end)
        # instructions += get_instructions_moves(pad, start, end) + 'A'
        new_instructions = get_instructions_moves(pad, start, end)
        new_instructionset = []
        for instruction in instructions:
            for new_instruction in new_instructions:
                new_instructionset.append(instruction + new_instruction + 'A')
            instructions = new_instructionset
    return list(set(instructions))

@cache
def get_instructions_batch(pad, code, size=8):
    code = 'A' + code
    instructions = [""]
    # print(code)
    for i in range(0, len(code), size):
        # print(code[i:i+size+1])
        new_instructions = get_instructions(pad, code[i:i+size+1])
        new_instructionset = []
        for instruction in instructions:
            for new_instruction in new_instructions:
                new_instructionset.append(instruction + new_instruction)
            instructions = new_instructionset
    return (list(set(instructions)))

@cache
def get_instructions_from_multiple(pad, codes):
    instr = []
    for code in codes:
        instr.extend(get_instructions_batch(pad, code))
    return instr
    
@cache
def get_complexity(code, intermediate_pads = 2):
    l1 = get_instructions_batch(numpad, code)
    for i in range(intermediate_pads):
        print(f"{i}/{intermediate_pads}", len(l1), len(l1[0]))
        l1 = get_instructions_from_multiple(dirpad, tuple(l1))
    shortest = min(map(len, l1))
    return shortest * int(code[:-1]), shortest, code

In [ ]:
l1 = get_instructions_batch(numpad, "286A")
l2 = get_instructions_from_multiple(dirpad, tuple(l1))
l3 = get_instructions_from_multiple(dirpad, tuple(l2))
l1, l2, l3

(['^<A^^Av>AvvA', '^<A^^A>vAvvA', '<^A^^Av>AvvA', '<^A^^A>vAvvA'],
 ['<Av<A>>^A<AA>Av<A>A^Av<AA>^A',
  '<Av<A>>^A<AA>AvA<A>^Av<AA>^A',
  'v<<A>^A>A<AA>Av<A>A^Av<AA>^A',
  'v<<A>^A>A<AA>AvA<A>^Av<AA>^A'],
 ['v<<A>>^Av<A<A>>^AvAA^<A>Av<<A>>^AAvA^Av<A<A>>^AvA^A<A>Av<A<A>>^AAvA^<A>A',
  'v<<A>>^Av<A<A>>^AvAA^<A>Av<<A>>^AAvA^Av<A>^Av<<A>>^AvA^<A>Av<A<A>>^AAvA^<A>A',
  'v<A<AA>>^AvA^<A>AvA^Av<<A>>^AAvA^Av<A<A>>^AvA^A<A>Av<A<A>>^AAvA^<A>A',
  'v<A<AA>>^AvA^<A>AvA^Av<<A>>^AAvA^Av<A>^Av<<A>>^AvA^<A>Av<A<A>>^AAvA^<A>A'])